In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../../nuclearpy_models')

In [2]:
from models.BE import dz_be, semf_be

In [3]:
exp_df = pd.read_csv('../../Data/Experimental/be_exp.csv', delimiter=',')

## Inference

In [4]:
exp_df.query("Z ==20 and  N == 20").BE

379    342.052184
Name: BE, dtype: float64

In [5]:
dz_be(20, 20) 

915.1800809893672

In [6]:
semf_be(10, 10)


160.1741423124985

In [7]:
exp_df["semf_pred"] = exp_df.apply(lambda x: semf_be(x["N"], x["Z"]), axis=1)
exp_df["dz_pred"] = exp_df.apply(lambda x: dz_be(x["N"], x["Z"]), axis=1)

In [8]:
np.zeros((19, 3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])